# 22.02.21 과제, violin plot

##### 참고
https://github.com/nowhyun/Study/blob/master/%EB%B0%95%EC%8A%A4_%ED%94%8C%EB%A1%AF(Box_Plot)%2C_%EB%B0%94%EC%9D%B4%EC%98%AC%EB%A6%BF_%ED%94%8C%EB%A1%AF(Violin_Plot).ipynb

violin그래프 그릴 때는 mean을 쓰지 않는다.

# top

RF top10
[51277, 51006, 50912, 50862, 63323026201,
 50983, 50882, 51003, 51274, 51275]

M1 entropy top10
[409606211, 74606211, 67434504, 54817525, 10019055302,
227194, 54858516, 74706811, 45006701, 4003822]

# Library

In [1]:
#데이터 분석 라이브러리
import numpy as np
print("numpy version: {}". format(np.__version__))

import pandas as pd

#시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(mpl.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#딥러닝 라이브러리
import tensorflow as tf

#기타 라이브러리
import os
import random
import time

#경고 에러 무시
import warnings
warnings.filterwarnings('ignore')
print('-'*50)

#시간 확인을 용이하게 해주는 라이브러리
from tqdm import tqdm

#결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

numpy version: 1.19.5
matplotlib version: 3.4.2
seaborn version: 0.11.1
--------------------------------------------------


# GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# CPU

In [3]:
#GPU 사용 설정, -1이면 CPU 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# data load

In [6]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# 2) MODEL
with tf.device('/device:GPU:0'):
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')
    
# 3) baseline
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 

    results = []
    preds = lstm2.predict(X_test)
    preds_mean = np.mean(preds) # base preds

    results.append({'feature':'BASELINE','baseline':preds_mean})
#     tf.device('/device:GPU:0').close()

In [7]:
results

[{'feature': 'BASELINE', 'baseline': 0.6369324}]

In [ ]:
# results 저장
results = [{'feature': 'BASELINE', 'baseline': 0.6369324}]

# all to 1

In [4]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# 2) MODEL
with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

# 3) all_to_1
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = X_test[:,:,k].copy()

        # all_to_1
        X_test[:,:,k] = 1

        # all_to_0
    #     X_test[:,:,k] = 0

        # inverse
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(X_test)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

#         loss_bce = bce(y_test, preds2).numpy()
#         results.append({'feature':features[k],'mean_preds2':np.mean(preds2)})
        X_test[:,:,k] = save_col

Using TensorFlow backend.
100%|██████████| 4068/4068 [30:14<00:00,  2.24it/s]


## Data processing

In [5]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_all_to_1 = df1.copy()
all_to_1_all_preds2 = preds2_dic.copy()

In [6]:
df_all_to_1.columns = ['mean_preds2']
# df_all_to_1

In [7]:
# df_all_to_1.to_csv('all_to_1_mean_preds2.csv', index = False)

In [11]:
pd.read_csv('all_to_1_mean_preds2.csv')

,mean_preds2
0,0.617006
1,0.610829
2,0.669560
3,0.603050
4,0.685346
...,...
4063,0.587123
4064,0.634779
4065,0.725083
4066,0.644183


## Violin plot으로 만들기 위한 전처리

In [8]:
for i,j in enumerate(all_to_1_all_preds2.keys()):
    if i == 0:
        ori = pd.Series(all_to_1_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(all_to_1_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기

In [9]:
# ori.to_csv('all_to_1_all_preds2.csv', index=False)

## Violin plot

### data load

In [12]:
data_all_to_1 = pd.read_csv('all_to_1_all_preds2.csv')

,0,50803,50804,50805,50806,50808,50809,50811,50813,50814,...,74300000533,74312000660,74312001370,76329301205,76336045517,78112001103,79511050204,87701071218,87701083336,87701089415
0,0.846375,0.828797,0.886364,0.818052,0.894676,0.868479,0.883605,0.889142,0.911049,0.812572,...,0.897870,0.872531,0.890084,0.857579,0.854523,0.791205,0.855482,0.913433,0.864324,0.872339
1,0.881663,0.860867,0.900111,0.855075,0.906826,0.887680,0.898340,0.902445,0.918431,0.851341,...,0.909656,0.891983,0.904220,0.881559,0.879545,0.839084,0.880150,0.921045,0.886565,0.891553
2,0.809148,0.789608,0.855021,0.774744,0.863110,0.832881,0.850730,0.856516,0.886325,0.769114,...,0.867722,0.834583,0.857071,0.815674,0.812697,0.748243,0.813302,0.889423,0.824123,0.834026
3,0.948484,0.947762,0.949039,0.947338,0.949946,0.949026,0.949573,0.949747,0.950726,0.947166,...,0.950035,0.949028,0.949689,0.948452,0.948374,0.946563,0.948421,0.950756,0.948679,0.949029
4,0.950014,0.949542,0.950909,0.949296,0.951106,0.950419,0.950814,0.950956,0.950759,0.949190,...,0.951166,0.950445,0.950933,0.950065,0.949992,0.948771,0.950025,0.951663,0.950215,0.950448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,0.698868,0.656243,0.778094,0.629322,0.793677,0.736258,0.769759,0.782063,0.836001,0.621491,...,0.803270,0.738057,0.781534,0.701614,0.697465,0.588300,0.698932,0.849694,0.716544,0.737101
1541,0.113465,0.110384,0.115172,0.110502,0.118206,0.113262,0.115121,0.116262,0.121820,0.110261,...,0.120161,0.115104,0.118156,0.113426,0.113208,0.109666,0.113280,0.127159,0.114476,0.114997
1542,0.747045,0.807824,0.875055,0.796489,0.884868,0.858360,0.873004,0.878650,0.904111,0.790904,...,0.888927,0.859530,0.880022,0.841800,0.838082,0.768621,0.839080,0.907172,0.849928,0.858971
1543,0.924180,0.917329,0.932129,0.914148,0.934119,0.927384,0.931208,0.932676,0.937470,0.912925,...,0.935196,0.928202,0.933013,0.924103,0.923438,0.907445,0.923687,0.939651,0.925951,0.928164


# all to 0

In [4]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# 2) MODEL
with tf.device('/device:GPU:0'):
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')
    
# 3) all_to_0
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = X_test[:,:,k].copy()

        # all_to_1
    #     X_test[:,:,k] = 1

        # all_to_0
        X_test[:,:,k] = 0

        # inverse
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
    #     X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(X_test)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

#         loss_bce = bce(y_test, preds2).numpy()
#         results.append({'feature':features[k],'bce':loss_bce})
        X_test[:,:,k] = save_col

Using TensorFlow backend.
100%|██████████| 4068/4068 [30:23<00:00,  2.23it/s]


## Data processing

In [5]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_all_to_0 = df1.copy()
all_to_0_all_preds2 = preds2_dic.copy()

# base_value = df1.iloc[0,1]
# df_all_best['baseline_bce'] = base_value
# # df_all_best
# df_all_best['diff'] = df_all_best['bce']-df_all_best['baseline_bce']

# df_all_best = df_all_best.drop(index=[0])
# df_all_best = df_all_best.sort_values(by=['diff'])

# #######################################################################

# diffdf = df_all_best[['feature','diff']]
# diffdf = diffdf.sort_values(by='diff')

In [6]:
df_all_to_0
all_to_0_all_preds2

,0
0,0.677977
1,0.636985
2,0.634996
3,0.636940
4,0.636746
...,...
4063,0.637362
4064,0.636932
4065,0.636043
4066,0.636931


{0: array([[0.8934177 ],
        [0.910596  ],
        [0.86493754],
        ...,
        [0.8417419 ],
        [0.9354465 ],
        [0.84076506]], dtype=float32),
 50803: array([[0.8576751 ],
        [0.8816626 ],
        [0.81608653],
        ...,
        [0.8417419 ],
        [0.9241805 ],
        [0.829637  ]], dtype=float32),
 50804: array([[0.8576751 ],
        [0.8816626 ],
        [0.81608653],
        ...,
        [0.8417419 ],
        [0.9241805 ],
        [0.8123625 ]], dtype=float32),
 50805: array([[0.8576751 ],
        [0.8816626 ],
        [0.81608653],
        ...,
        [0.8417419 ],
        [0.9241805 ],
        [0.829637  ]], dtype=float32),
 50806: array([[0.8576751 ],
        [0.8816626 ],
        [0.81608653],
        ...,
        [0.8417419 ],
        [0.9241805 ],
        [0.829637  ]], dtype=float32),
 50808: array([[0.8576751 ],
        [0.8816626 ],
        [0.81608653],
        ...,
        [0.84656435],
        [0.9241805 ],
        [0.829637  ]], dtype=

In [9]:
df_all_to_0.columns = ['mean_preds2']
# df_all_to_0

In [14]:
# df_all_to_0.to_csv('all_to_0_mean_preds2.csv', index = False)

In [15]:
# 확인 cell
pd.read_csv('all_to_0_mean_preds2.csv')

,mean_preds2
0,0.677977
1,0.636985
2,0.634996
3,0.636940
4,0.636746
...,...
4063,0.637362
4064,0.636932
4065,0.636043
4066,0.636931


## Violin plot으로 만들기 위한 전처리

In [34]:
# pd.DataFrame(all_to_0_all_preds2)
for i,j in enumerate(all_to_0_all_preds2.keys()):
#     len(all_to_0_all_preds2[j])
#     all_to_0_all_preds2[j].flatten()
    if i == 0:
        ori = pd.Series(all_to_0_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(all_to_0_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기
#     break

In [ ]:
# all_to_0_all_preds2.keys()

In [36]:
# ori.to_csv('all_to_0_all_preds2.csv', index=False)

## violin plot 그리기

### data load

In [39]:
data = pd.read_csv('all_to_0_all_preds2.csv')

In [61]:
data
data['50803']

,0,50803,50804,50805,50806,50808,50809,50811,50813,50814,...,74300000533,74312000660,74312001370,76329301205,76336045517,78112001103,79511050204,87701071218,87701083336,87701089415
0,0.893418,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,...,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675,0.857675
1,0.910596,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,...,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663,0.881663
2,0.864938,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,...,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087,0.816087
3,0.950064,0.948484,0.947645,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,...,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484,0.948484
4,0.951145,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.948576,0.950073,...,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073,0.950073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,0.802730,0.702804,0.702217,0.702804,0.702804,0.702804,0.702804,0.702804,0.692118,0.702804,...,0.702804,0.702804,0.702804,0.702804,0.702804,0.702804,0.702804,0.702804,0.702804,0.702804
1541,0.121324,0.113465,0.113465,0.113465,0.113465,0.113609,0.113465,0.113465,0.113465,0.113465,...,0.113465,0.113465,0.113465,0.113465,0.113465,0.113465,0.113465,0.113465,0.113465,0.113465
1542,0.841742,0.841742,0.841742,0.841742,0.841742,0.846564,0.843166,0.842124,0.841742,0.841742,...,0.841742,0.841742,0.841742,0.841742,0.841742,0.841742,0.841742,0.841742,0.841742,0.841742
1543,0.935446,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.920908,0.924180,...,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180,0.924180


0       0.857675
1       0.881663
2       0.816087
3       0.948484
4       0.950073
          ...   
1540    0.702804
1541    0.113465
1542    0.841742
1543    0.924180
1544    0.829637
Name: 50803, Length: 1545, dtype: float64

RF top10
[51277, 51006, 50912, 50862, 63323026201,
 50983, 50882, 51003, 51274, 51275]

M1 entropy top10
[409606211, 74606211, 67434504, 54817525, 10019055302,
227194, 54858516, 74706811, 45006701, 4003822]

In [65]:
# 스타일 설정
sns.set_style('whitegrid')

# 사용할 특성 2가지 선택
hue_col = [0,1]
x_col = '51277' #문자로 들어있음.
y_col = '51277'

#데이터 확인
x_col_val = data[x_col].values
y_col_val = data[y_col].values
hue_col_val = [0,1]#data[hue_col].values
print('{}행의 값:\n{}'.format(x_col, x_col_val))
print('{}행의 값:\n{}'.format(y_col, y_col_val))
print('{}행의 값:\n{}'.format(hue_col, hue_col_val))

51277행의 값:
[0.8576751  0.7007793  0.6259553  ... 0.8417419  0.87259173 0.829637  ]
51277행의 값:
[0.8576751  0.7007793  0.6259553  ... 0.8417419  0.87259173 0.829637  ]
[0, 1]행의 값:
[0, 1]


# inverse

In [13]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# 2) MODEL
with tf.device('/device:GPU:0'):
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

# 3) inverse
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = X_test[:,:,k].copy()

        # all_to_1
    #     X_test[:,:,k] = 1

        # all_to_0
    #     X_test[:,:,k] = 0

        # inverse
        X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
        X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
        X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(X_test)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

#         loss_bce = bce(y_test, preds2).numpy()
#         results.append({'feature':features[k],'bce':loss_bce})
        X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [30:18<00:00,  2.24it/s]


## Data processing

In [14]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_inverse = df1.copy()
inverse_all_preds2 = preds2_dic.copy()

In [15]:
df_inverse.columns = ['mean_preds2']
# df_inverse

In [16]:
# df_inverse.to_csv('inverse_mean_preds2.csv', index = False)

In [20]:
pd.read_csv('inverse_mean_preds2.csv')

,mean_preds2
0,0.658366
1,0.610879
2,0.667633
3,0.603060
4,0.685195
...,...
4063,0.587544
4064,0.634779
4065,0.724196
4066,0.644181


## Violin plot으로 만들기 위한 전처리

In [17]:
for i,j in enumerate(inverse_all_preds2.keys()):
    if i == 0:
        ori = pd.Series(inverse_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(inverse_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기

In [18]:
# ori.to_csv('inverse_all_preds2.csv', index=False)

## Violin plot

### data load

In [19]:
data_inverse = pd.read_csv('inverse_all_preds2.csv')

In [21]:
data_inverse

,0,50803,50804,50805,50806,50808,50809,50811,50813,50814,...,74300000533,74312000660,74312001370,76329301205,76336045517,78112001103,79511050204,87701071218,87701083336,87701089415
0,0.887100,0.828797,0.886364,0.818052,0.894676,0.868479,0.883605,0.889142,0.911049,0.812572,...,0.897870,0.872531,0.890084,0.857579,0.854523,0.791205,0.855482,0.913433,0.864324,0.872339
1,0.910596,0.860867,0.900111,0.855075,0.906826,0.887680,0.898340,0.902445,0.918431,0.851341,...,0.909656,0.891983,0.904220,0.881559,0.879545,0.839084,0.880150,0.921045,0.886565,0.891553
2,0.860527,0.789608,0.855021,0.774744,0.863110,0.832881,0.850730,0.856516,0.886325,0.769114,...,0.867722,0.834583,0.857071,0.815674,0.812697,0.748243,0.813302,0.889423,0.824123,0.834026
3,0.950064,0.947762,0.948257,0.947338,0.949946,0.949026,0.949573,0.949747,0.950726,0.947166,...,0.950035,0.949028,0.949689,0.948452,0.948374,0.946563,0.948421,0.950756,0.948679,0.949029
4,0.951099,0.949542,0.950909,0.949296,0.951106,0.950419,0.950814,0.950956,0.949469,0.949190,...,0.951166,0.950445,0.950933,0.950065,0.949992,0.948771,0.950025,0.951663,0.950215,0.950448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,0.799976,0.656243,0.776968,0.629322,0.793677,0.736258,0.769759,0.782063,0.830687,0.621491,...,0.803270,0.738057,0.781534,0.701614,0.697465,0.588300,0.698932,0.849694,0.716544,0.737101
1541,0.121324,0.110384,0.115172,0.110502,0.118206,0.113395,0.115121,0.116262,0.121820,0.110261,...,0.120161,0.115104,0.118156,0.113426,0.113208,0.109666,0.113280,0.127159,0.114476,0.114997
1542,0.747045,0.807824,0.875055,0.796489,0.884868,0.862500,0.874027,0.878937,0.904111,0.790904,...,0.888927,0.859530,0.880022,0.841800,0.838082,0.768621,0.839080,0.907172,0.849928,0.858971
1543,0.935446,0.917329,0.932129,0.914148,0.934119,0.927384,0.931208,0.932676,0.936004,0.912925,...,0.935196,0.928202,0.933013,0.924103,0.923438,0.907445,0.923687,0.939651,0.925951,0.928164
